In [16]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.1 MB 11.3 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.1 MB 22.9 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 24.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/44.5 MB ? eta -:--:--
   ----- ---------------------------------- 6.3/44.5 MB 32.2 MB/s eta 0:00:02
   ----------- ---------------------------- 13.1/44.5 MB 31.6 MB/s eta 0:00:01
   ------------------ --------------------- 20.2/44.5 MB 33.5 MB/s eta 0:00:01
   ------------------------ --------------- 27.5/44.5 MB 34.2 MB/s eta 0:00:01
   ------------------------------- -------- 34.6/44.5 MB 33.8 MB/s eta 0:00:01
   ------------------------------------- -- 41.7/44.5 MB 34.4 MB/s eta 0:00:01
   -----

In [94]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

In [142]:
boxes = pd.read_csv('box_21_24.csv')

poss = pd.read_csv('poss_21_24.csv')

C:\Users\willi\AppData\Local\Temp\ipykernel_49260\3946115115.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  boxes = pd.read_csv('box_21_24.csv')


In [72]:
boxes_ids_years

,game_id,season
0,401370398,2021
22,401370397,2021
44,401370396,2021
66,401370395,2021
88,401370442,2021
...,...,...
21549,401620222,2024
21569,401620220,2024
21590,401620221,2024
21613,401620219,2024


In [144]:
boxes_ids_years

,minutes
athlete_id,
91.0,1955.0
585.0,3550.0
689.0,464.0
764.0,2116.0
766.0,1831.0
...,...
5017726.0,831.0
5105752.0,139.0
5106150.0,176.0


In [145]:
# poss = poss.drop(columns=['Unnamed: 0'])
boxes_ids_years = boxes[['game_id', 'season']].drop_duplicates()
poss = poss.merge(boxes_ids_years, on=['game_id'])

In [148]:
boxes_minutes = boxes[['game_id', 'season', 'minutes', 'athlete_id','athlete_display_name']].groupby(['athlete_id','season'], as_index=False).agg({'minutes':'sum'})
boxes_minutes

,athlete_id,season,minutes
0,91.0,2021,908.0
1,91.0,2022,1047.0
2,585.0,2021,758.0
3,585.0,2022,961.0
4,585.0,2023,709.0
...,...,...,...
643,5105752.0,2024,139.0
644,5106150.0,2024,176.0
645,5130672.0,2023,461.0
646,5130672.0,2024,267.0


In [78]:
player_cols = [
    'off1', 'off2', 'off3', 'off4', 'off5',
    'def1', 'def2', 'def3', 'def4', 'def5'
]

In [105]:
def build_player_list(poss):
    players = list(
        set(list(poss['off1'].unique()) + list(poss['off2'].unique()) + 
            list(poss['off3']) + list(poss['off4'].unique()) + 
            list(poss['off5'].unique()) + list(poss['def1'].unique()) + 
            list(poss['def2'].unique()) + list(poss['def3'].unique()) + 
            list(poss['def4'].unique()) + list(poss['def5'].unique())))
    players.sort()
    return players



def map_players(row_in, players):
    p1 = row_in[0]
    p2 = row_in[1]
    p3 = row_in[2]
    p4 = row_in[3]
    p5 = row_in[4]
    p6 = row_in[5]
    p7 = row_in[6]
    p8 = row_in[7]
    p9 = row_in[8]
    p10 = row_in[9]

    rowOut = np.zeros([len(players) * 2])

    rowOut[players.index(p1)] = 1
    rowOut[players.index(p2)] = 1
    rowOut[players.index(p3)] = 1
    rowOut[players.index(p4)] = 1
    rowOut[players.index(p5)] = 1

    rowOut[players.index(p6) + len(players)] = -1
    rowOut[players.index(p7) + len(players)] = -1
    rowOut[players.index(p8) + len(players)] = -1
    rowOut[players.index(p9) + len(players)] = -1
    rowOut[players.index(p10) + len(players)] = -1

    return rowOut

def convert_to_matricies(possessions, name, players):
    # extract only the columns we need

    # Convert the columns of player ids into a numpy matrix
    stints_x_base = possessions[player_cols].to_numpy()
    # Apply our mapping function to the numpy matrix
    stint_X_rows = np.apply_along_axis(map_players, 1, stints_x_base, players)

    # Convert the column of target values into a numpy matrix
    stint_Y_rows = possessions[name].to_numpy()

    # return matricies and possessions series
    return stint_X_rows, stint_Y_rows

# Convert lambda value to alpha needed for ridge CV
def lambda_to_alpha(lambda_value, samples):
    return (lambda_value * samples) / 2.0

# Convert RidgeCV alpha back into a lambda value
def alpha_to_lambda(alpha_value, samples):
    return (alpha_value * 2.0) / samples

def calculate_rapm(train_x, train_y, lambdas, name, players):
    # convert our lambdas to alphas
    alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas]

    # create a 5 fold CV ridgeCV model. Our target data is not centered at 0, so we want to fit to an intercept.
    clf = RidgeCV(alphas=alphas, cv=5, fit_intercept=True)

    # fit our training data
    model = clf.fit(train_x, train_y,)

    # convert our list of players into a mx1 matrix
    player_arr = np.transpose(np.array(players).reshape(1, len(players)))

    # extract our coefficients into the offensive and defensive parts
    coef_offensive_array = model.coef_[0:len(players)][np.newaxis].T
    coef_defensive_array = model.coef_[len(players):][np.newaxis].T

    # concatenate the offensive and defensive values with the playey ids into a mx3 matrix
    player_id_with_coef = np.concatenate([player_arr, coef_offensive_array, coef_defensive_array], axis=1)
    # build a dataframe from our matrix
    players_coef = pd.DataFrame(player_id_with_coef)
    intercept = model.intercept_

    # apply new column names
    players_coef.columns = ['playerId', 'O{0}'.format(name),         
    'D{0}'.format(name)]

    # Add the offesnive and defensive components together (we should really be weighing this to the number of offensive and defensive possession played as they are often not equal).
    players_coef[name] = players_coef['O{0}'.format(name)] + players_coef['D{0}'.format(name)]

    # rank the values
    players_coef['{0} Rank'.format(name)] = players_coef[name].rank(ascending=False)
    players_coef['O{0} Rank'.format(name)] = players_coef[         
        'O{0}'.format(name)].rank(ascending=False)
    players_coef['D{0} Rank'.format(name)] = players_coef[
        'D{0}'.format(name)].rank(ascending=False)

    return players_coef, intercept


def calculate_rapm_with_decay(possessions, name, lambdas_rapm, players, current_year=2023, decay_factor=0.9):
    """
    possessions: DataFrame with columns including 'season', player_cols (off1..def5), and target column `name`.
    name: the target variable (e.g., 'PPP', 'points_scored')
    lambdas_rapm: list of lambdas to try (e.g., [0.01, 0.05, 0.1])
    players: list of all players
    current_year: int, the reference year from which decay is computed
    decay_factor: float, exponential decay per season (e.g., 0.9 means 10% downweight per year older)
    """

    # Compute weights based on how old the season is
    # This assumes possessions has a 'season' column
    possessions['years_ago'] = current_year - possessions['season']
    possessions['weight'] = decay_factor ** possessions['years_ago']

    # Convert possessions to X and y
    train_x, train_y = convert_to_matricies(possessions, name, players)
    sample_weights = possessions['weight'].values

    # Convert lambdas to alphas
    alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas_rapm]

    # We'll do manual cross-validation to pick the best alpha
    # For RAPM, a common scoring metric might be MSE or R^2.
    # Here we'll use R^2 for simplicity, but you could use neg_mean_squared_error too.
    best_alpha = None
    best_score = -np.inf
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    for alpha in alphas:
        cv_scores = []
        for train_index, val_index in kf.split(train_x):
            X_train_fold, X_val_fold = train_x[train_index], train_x[val_index]
            y_train_fold, y_val_fold = train_y[train_index], train_y[val_index]
            w_train_fold = sample_weights[train_index]

            model = Ridge(alpha=alpha, fit_intercept=True)
            model.fit(X_train_fold, y_train_fold, sample_weight=w_train_fold)
            score = model.score(X_val_fold, y_val_fold)
            cv_scores.append(score)

        avg_score = np.mean(cv_scores)
        if avg_score > best_score:
            best_score = avg_score
            best_alpha = alpha

    # Now fit the final model with the best alpha
    final_model = Ridge(alpha=best_alpha, fit_intercept=True)
    final_model.fit(train_x, train_y, sample_weight=sample_weights)

    # Extract coefficients similarly to the original code
    player_arr = np.transpose(np.array(players).reshape(1, len(players)))

    coef_offensive_array = final_model.coef_[0:len(players)][np.newaxis].T
    coef_defensive_array = final_model.coef_[len(players):][np.newaxis].T

    player_id_with_coef = np.concatenate([player_arr, coef_offensive_array, coef_defensive_array], axis=1)
    players_coef = pd.DataFrame(player_id_with_coef, columns=['playerId', 'ORAPM', 'DRAPM'])
    players_coef[name] = players_coef['ORAPM'] + players_coef['DRAPM']

    players_coef['RAPM Rank'] = players_coef[name].rank(ascending=False)
    players_coef['ORAPM Rank'] = players_coef['ORAPM'].rank(ascending=False)
    players_coef['DRAPM Rank'] = players_coef['DRAPM'].rank(ascending=False)

    intercept = final_model.intercept_

    return players_coef, intercept, best_alpha



In [38]:
poss.dtypes

game_id          int64
possession_id    int64
off1             int64
off2             int64
off3             int64
off4             int64
off5             int64
def1             int64
def2             int64
def3             int64
def4             int64
def5             int64
points_scored    int64
PPP              int64
dtype: object

In [83]:
player_list = build_player_list(poss)

In [84]:
poss['PPP'] = poss['points_scored'] * 100

In [85]:
train_x, train_y = convert_to_matricies(poss, 'PPP', player_list)

In [86]:
lambdas_rapm = [.01, .05, .1]

In [122]:
results, intercept, best_alpha = calculate_rapm_with_decay(poss, 'PPP', lambdas_rapm,  player_list)

In [135]:
pd.set_option('display.max_columns', 500)

,Unnamed: 0,game_id,season,season_type,game_date,game_date_time,athlete_id,athlete_display_name,team_id,team_name,team_location,team_short_display_name,minutes,field_goals_made,field_goals_attempted,three_point_field_goals_made,three_point_field_goals_attempted,free_throws_made,free_throws_attempted,offensive_rebounds,defensive_rebounds,rebounds,assists,steals,blocks,turnovers,fouls,plus_minus,points,starter,ejected,did_not_play,reason,active,athlete_jersey,athlete_short_name,athlete_headshot_href,athlete_position_name,athlete_position_abbreviation,team_display_name,team_uid,team_slug,team_logo,team_abbreviation,team_color,team_alternate_color,home_away,team_winner,team_score,opponent_team_id,opponent_team_name,opponent_team_location,opponent_team_display_name,opponent_team_abbreviation,opponent_team_logo,opponent_team_color,opponent_team_alternate_color,opponent_team_score
0,1,401370398,2021,3,2021-10-17,2021-10-17 15:11:00,3142086.0,Brianna Turner,11,Mercury,Phoenix,Mercury,36.0,2.0,6.0,0.0,0.0,1.0,1.0,5.0,7.0,12.0,2.0,2.0,2.0,3.0,4.0,-9,5.0,True,False,False,COACH'S DECISION,True,21.0,B. Turner,https://a.espncdn.com/i/headshots/wnba/players...,Forward,F,Phoenix Mercury,s:40~l:59~t:11,phoenix-mercury,https://a.espncdn.com/i/teamlogos/wnba/500/phx...,PHX,3a267d,e56020,away,False,74,19,Sky,Chicago,Chicago Sky,CHI,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,4f91cd,f1f2f3,80
1,2,401370398,2021,3,2021-10-17,2021-10-17 15:11:00,2490553.0,Brittney Griner,11,Mercury,Phoenix,Mercury,38.0,12.0,19.0,0.0,1.0,4.0,4.0,0.0,7.0,7.0,2.0,1.0,1.0,0.0,0.0,-4,28.0,True,False,False,COACH'S DECISION,True,42.0,B. Griner,https://a.espncdn.com/i/headshots/wnba/players...,Center,C,Phoenix Mercury,s:40~l:59~t:11,phoenix-mercury,https://a.espncdn.com/i/teamlogos/wnba/500/phx...,PHX,3a267d,e56020,away,False,74,19,Sky,Chicago,Chicago Sky,CHI,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,4f91cd,f1f2f3,80
2,3,401370398,2021,3,2021-10-17,2021-10-17 15:11:00,585.0,Diana Taurasi,11,Mercury,Phoenix,Mercury,33.0,4.0,16.0,3.0,9.0,5.0,6.0,0.0,2.0,2.0,2.0,0.0,1.0,3.0,4.0,-22,16.0,True,False,False,COACH'S DECISION,True,3.0,D. Taurasi,https://a.espncdn.com/i/headshots/wnba/players...,Guard,G,Phoenix Mercury,s:40~l:59~t:11,phoenix-mercury,https://a.espncdn.com/i/teamlogos/wnba/500/phx...,PHX,3a267d,e56020,away,False,74,19,Sky,Chicago,Chicago Sky,CHI,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,4f91cd,f1f2f3,80
3,4,401370398,2021,3,2021-10-17,2021-10-17 15:11:00,1107.0,Shey Peddy,11,Mercury,Phoenix,Mercury,18.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,4.0,4.0,3.0,1.0,0.0,1.0,1.0,-11,0.0,True,False,False,COACH'S DECISION,True,11.0,S. Peddy,https://a.espncdn.com/i/headshots/wnba/players...,Guard,G,Phoenix Mercury,s:40~l:59~t:11,phoenix-mercury,https://a.espncdn.com/i/teamlogos/wnba/500/phx...,PHX,3a267d,e56020,away,False,74,19,Sky,Chicago,Chicago Sky,CHI,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,4f91cd,f1f2f3,80
4,5,401370398,2021,3,2021-10-17,2021-10-17 15:11:00,2491205.0,Skylar Diggins-Smith,11,Mercury,Phoenix,Mercury,37.0,7.0,18.0,2.0,7.0,0.0,0.0,0.0,2.0,2.0,8.0,0.0,1.0,5.0,1.0,-2,16.0,True,False,False,COACH'S DECISION,True,4.0,S. Diggins-Smith,https://a.espncdn.com/i/headshots/wnba/players...,Guard,G,Phoenix Mercury,s:40~l:59~t:11,phoenix-mercury,https://a.espncdn.com/i/teamlogos/wnba/500/phx...,PHX,3a267d,e56020,away,False,74,19,Sky,Chicago,Chicago Sky,CHI,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,4f91cd,f1f2f3,80


In [124]:
results = np.round(results, decimals=2)
results = results.reindex(sorted(results.columns), axis=1)
boxes = boxes.drop_duplicates(subset=['athlete_id'])
results = results.merge(boxes[['athlete_id','athlete_display_name','team_name']], left_on='playerId', right_on='athlete_id', how='left')
results = results.rename(columns={'athlete_display_name': 'Player Name', 'PPP':'RAPM'})

In [118]:
results

,DRAPM,DRAPM Rank,ORAPM,ORAPM Rank,RAPM,RAPM Rank,playerId,athlete_id,Player Name
0,-0.43,206.0,1.90,12.0,1.47,21.0,91.0,91.0,Sue Bird
1,-1.30,252.0,1.19,28.0,-0.11,133.0,585.0,585.0,Diana Taurasi
2,-0.33,193.0,-0.76,220.0,-1.09,235.0,689.0,689.0,Candice Dupree
3,-0.40,203.0,1.65,18.0,1.25,27.0,764.0,764.0,Allie Quigley
4,-0.54,216.0,0.27,65.0,-0.27,159.0,766.0,766.0,Sylvia Fowles
...,...,...,...,...,...,...,...,...,...
255,0.93,21.0,-0.61,210.0,0.32,69.0,5017726.0,5017726.0,Jade Melbourne
256,0.28,80.0,-0.13,129.0,0.15,84.0,5105752.0,5105752.0,Alissa Pili
257,-0.21,175.0,-0.31,168.0,-0.51,185.0,5106150.0,5106150.0,Stephanie Soares
258,-0.89,239.0,-0.62,213.0,-1.51,251.0,5130672.0,5130672.0,Ivana Dojkic


In [125]:
results.sort_values(by='RAPM', ascending=False).head(50)

,DRAPM,DRAPM Rank,ORAPM,ORAPM Rank,RAPM,RAPM Rank,playerId,athlete_id,Player Name,team_name
73,0.27,84.0,4.15,1.0,4.42,1.0,2998928.0,2998928.0,Breanna Stewart,Storm
36,1.64,3.0,2.02,11.0,3.66,2.0,2491205.0,2491205.0,Skylar Diggins-Smith,Mercury
96,0.26,86.0,3.20,2.0,3.46,3.0,3065570.0,3065570.0,Kelsey Plum,Aces
6,1.09,16.0,2.29,5.0,3.38,4.0,812.0,812.0,Candace Parker,Sky
76,1.29,8.0,1.69,17.0,2.98,5.0,2999101.0,2999101.0,Jonquel Jones,Sun
38,1.29,7.0,1.55,20.0,2.85,6.0,2491877.0,2491877.0,Elena Delle Donne,Mystics
40,0.53,43.0,2.03,10.0,2.55,7.0,2529122.0,2529122.0,Chelsea Gray,Aces
248,1.75,1.0,0.78,39.0,2.53,8.0,4683006.0,4683006.0,Leonie Fiebich,Liberty
91,1.26,10.0,1.19,27.0,2.46,9.0,3058895.0,3058895.0,Brionna Jones,Sun
11,0.51,45.0,1.88,14.0,2.40,10.0,869.0,869.0,DeWanna Bonner,Sun


In [127]:
results.groupby('team_name').agg({'RAPM':'sum'}).sort_values(by='RAPM', ascending=False)

,RAPM
team_name,
Sun,11.66
Aces,6.55
Sky,4.63
Lynx,4.49
Mystics,2.36
Storm,2.29
Mercury,-0.83
Liberty,-4.14
Wings,-4.15
